In [ ]:
!pip install flask pyngrok sentence-transformers pillow


In [ ]:
# !pip install flask pyngrok sentence-transformers pillow
from flask import Flask, request, jsonify
from pyngrok import ngrok
import json
from sentence_transformers import SentenceTransformer
from PIL import Image
from google.colab import userdata


model = SentenceTransformer('clip-ViT-B-32')

def getEmbeddingFromFile(file):
    image = Image.open(file.stream)
    embedding = model.encode(image)
    return embedding.tolist()



ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))

app = Flask(__name__)

@app.route("/test", methods = ["GET"])
def test():
    return jsonify({"status": "working"})


@app.route("/upload", methods = ["POST"])
def upload():
  retur = []
  imageEmbeds = []
  textEmbed= []
  for img in request.files.getlist("images"):
    imageEmbeds.append(getEmbeddingFromFile(img))
  prompt = request.form["text"]:
  textEmbed = model.encode(prompt).tolist()
  retur = 0.7*imageEmbeds + 0.3*textEmbed
  return jsonify(retur), 200


if __name__ == "__main__":
  public_url = ngrok.connect(5000)
  print("Ngrok tunnel URL:", public_url)
  app.run(host='0.0.0.0' , port=5000, debug = False)
